In [15]:
## Imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim  as optim
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
from torchvision.transforms import transforms

In [16]:
## Create Fully connected network

class NN(nn.Module):
    
    def __init__(self,input_shape,output_shape):
        super(NN,self).__init__()
        self.fc1 = nn.Linear(input_shape,50)
        self.fc2 = nn.Linear(50,output_shape)
        
    def forward(self,x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [17]:
# set device
device = torch.device('cuda' if torch.cuda.is_available() else 'mps')
device

device(type='cuda')

In [24]:
# Hyperparameters
input_shape = 28*28
output_shape = 10
batch = 100
num_epoch = 50
learning_rate = 0.01

In [25]:
## load Dataset
train_dataset = datasets.MNIST(root = 'datasets/',train=True,download=True,transform = transforms.ToTensor())
train_loader = DataLoader(dataset=train_dataset,batch_size=batch,shuffle=True)

test_dataset = datasets.MNIST(root = 'datasets/',train=False,download=True,transform = transforms.ToTensor())
test_loader = DataLoader(dataset=test_dataset,batch_size=batch,shuffle=True)

In [26]:
## Initialize model 
model = NN(input_shape,output_shape).to(device)

In [27]:
## Losss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=learning_rate)

In [28]:
data, target = next(iter(train_loader))

In [29]:
%%time
## Train Network 
for epoch in range(num_epoch):
    losses = []
    
    for batch_idx, (data,target) in enumerate(train_loader):
        # Get data to cuda 
        data = data.to(device=device)
        target =target.to(device=device)
        
        # reshape
        data = data.view(data.shape[0],-1)
        
        # forward
        scores = model(data)
        loss = criterion(scores,target)
        losses.append(loss.item())
        
        # backward
        optimizer.zero_grad()
        loss.backward()
        
        # gradient descent
        optimizer.step()
        
    print(f"{epoch} --> loss {sum(losses)/len(losses)}")

0 --> loss 0.2536257966669897
1 --> loss 0.13469973117423553
2 --> loss 0.11098148941372832
3 --> loss 0.09285901881288737
4 --> loss 0.08967291217840587
5 --> loss 0.08508636639996742
6 --> loss 0.07677940334484447
7 --> loss 0.07392352725786623
8 --> loss 0.06192851935435707
9 --> loss 0.06461279439448844
10 --> loss 0.06805265597315156
11 --> loss 0.059979464348628726
12 --> loss 0.06053587376283152
13 --> loss 0.059683820935315456
14 --> loss 0.059815789360606386
15 --> loss 0.053127331994473934
16 --> loss 0.055695770989320104
17 --> loss 0.060135870669400904
18 --> loss 0.04966364919809469
19 --> loss 0.04323818794646892
20 --> loss 0.04591647792730327
21 --> loss 0.04640824839231451
22 --> loss 0.045094738269302374
23 --> loss 0.04589871974544015
24 --> loss 0.0401137935013882
25 --> loss 0.04914994327487875
26 --> loss 0.046205610118957643
27 --> loss 0.042044225214317145
28 --> loss 0.0436537388135495
29 --> loss 0.03978866477619012
30 --> loss 0.040702597907273534
31 --> loss

In [12]:
# check the accuracy of out trained model 
def check_accuracy(loader,model):
    for data,target in loader:
        num_correct = 0
        num_sample = 0
        model.eval()
        with torch.no_grad():
            data = data.to(device=device)
            target = target.to(device=device)

            # reshape 
            data = data.view(data.shape[0],-1)


            scores = model(data)
            _, pred = scores.max(1)
            # print(list(zip(pred,target)))
            num_correct += sum(pred == target)
            num_sample  += pred.shape[0]
    print(f'Total {num_correct} correct  / out of {num_sample} - accuracy {num_correct/num_sample :.3f} ')

  

In [13]:
# on test dataset
check_accuracy(test_loader,model)

Total 95 correct  / out of 100 - accuracy 0.950 


In [14]:
# on train datasets
check_accuracy(train_loader,model)

Total 98 correct  / out of 100 - accuracy 0.980 
